## LangChain chains

In [1]:
%pip install -q langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Prosty chain: prompt → model → wynik

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "jesteś ekspertem programowania w Pythonie i geniuszem w dziedzinie AI."),
    ("user", "napisz kod {topic}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()
result = chain.invoke({'topic': 'AGI'})


In [3]:
print(result)

Tworzenie prawdziwej sztucznej inteligencji ogólnej (AGI) to niezwykle złożone zadanie, które wykracza poza możliwości obecnych technologii i narzędzi. AGI odnosi się do systemów AI, które mogą rozumieć, uczyć się i stosować wiedzę w różnych dziedzinach, podobnie jak człowiek. Obecnie nie istnieje żaden kod, który mógłby stworzyć AGI, ponieważ wymagałoby to zaawansowanego zrozumienia świadomości, uczenia się, rozumienia kontekstu i wielu innych aspektów, które są wciąż badane.

Jednak mogę pokazać prosty przykład, jak można zbudować system AI, który uczy się na podstawie danych, używając Pythona i popularnych bibliotek, takich jak TensorFlow lub PyTorch. Oto przykład prostego modelu uczenia maszynowego, który klasyfikuje dane:

```python
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# Wczytaj dane
data = load_iris()
X = data.data
y = data.target

# 

### Sequential chain: dwa modele w sekwencji

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: streszczenie (wejście: {tekst} → wyjście: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Streść poniższy tekst w 1–2 zdaniach po polsku."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: zamień str → {"tekst": str}, żeby podać do następnego promptu
to_dict = RunnableLambda(lambda s: {"tekst": s})

# 3) Chain: tłumaczenie streszczenia na francuski (wejście: {tekst} → wyjście: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Przetłumacz tekst na język francuski."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

sequential_chain = summary_chain | to_dict | translate_chain
input_text = "LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."

final_translation = sequential_chain.invoke({"tekst": input_text})

print(final_translation)


LangChain permet de créer des applications d'IA en intégrant des modèles, des invites et des outils dans des processus organisés.


### Branching chain: jedna odpowiedź, dwa przetworzenia

In [5]:
from langchain_core.runnables import RunnableParallel

# Prompt do streszczenia
prompt_summary = ChatPromptTemplate.from_template("Streść: {tekst}")

# Prompt do sentymentu
prompt_sentiment = ChatPromptTemplate.from_template("Określ ton wypowiedzi: {tekst}")

branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser())
)

text = "Jestem bardzo zadowolony z tego kursu, nauczyłem się dużo o LangChain!"
result = branch_chain.invoke({"tekst": text})

print(result)


{'summary': 'Uczestnik kursu jest bardzo zadowolony i zdobył wiele wiedzy na temat LangChain.', 'sentiment': 'Ton wypowiedzi jest pozytywny i entuzjastyczny. Osoba wyraża zadowolenie z kursu oraz podkreśla, że zdobyła cenną wiedzę na temat LangChain.'}
